In [169]:
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI
from langchain.chains import LLMChain
import textwrap
from langchain.chains import LLMSummarizationCheckerChain
import os
from PyPDF2 import PdfReader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA

os.environ["OPENAI_API_KEY"] = #INSERT YOUR KEY HERE#





def read_PDF(pdf_dir):
    doc_reader = PdfReader(pdf_dir)
    raw_text = ''
    for i, page in enumerate(doc_reader.pages):
        text = page.extract_text()
        if text:
            raw_text += text
    return raw_text

def split_text(raw_text):
    text_splitter = CharacterTextSplitter(        
        separator = "\n",
        chunk_size = 2500,
        chunk_overlap  = 400, #striding over the text
        length_function = len,
    )
    texts = text_splitter.split_text(raw_text)
    return texts

raw_text = read_PDF(r"C:\Users\felip\OneDrive - Fundacao Getulio Vargas - FGV\!_Paper04_Chat_PDF\fact_check_linha_35.pdf")
texts = split_text(raw_text)

inconsistencias = """Errors in the classification of expenses in the Code of Application
Discrepancies in the values questioned regarding expenses related to education
Divergences in Restos a Pagar values
Divergences between the SIOPE and SGRES Contábil regarding maintenance and development expenses
Error in the accounting of expenses for Actions and Public Health Services
Inconsistencies in the Portal of Transparency, including missing information and outdated contracts
Incomplete and non-compliant Demonstrativo de Disponibilidade de Caixa e Restos a Pagar
Deficiencies in the submission of required documents and reports
Deficit in revenue compared to the budgeted amount
Decrease in tax revenue from the previous year
Budgetary savings due to underutilization of funds
Superavit in the comparison of revenue and expenses
Delays in the submission of budgetary documents and reports to the TCE-PI"""

inconsistencias = inconsistencias.split('\n')
#for i in inconsistencias:
#    query = i
#    print(query)

inconsistence_check = """Role: You are a fact checker.
You know this context:{context}
Here is a fact: {question}
Task: determine whether the fact is true or false about the subject.
If you are unable to determine whether the fact is true or false, output "Undetermined".
If the fact is true or false, explain why.
Here is the answer:"""

prompt_inconsistence_check = PromptTemplate(template=inconsistence_check, input_variables=['context', 'question'])

embeddings = OpenAIEmbeddings()
#docsearch = FAISS.from_texts(texts, embeddings)      
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})
rqa = RetrievalQA.from_chain_type(llm=OpenAI(), 
                                  chain_type="stuff", 
                                  retriever=retriever,
                                  chain_type_kwargs={"prompt": prompt_inconsistence_check},
                                  return_source_documents=True)


In [167]:
inconsistencias[10]

'Budgetary savings due to underutilization of funds'

In [174]:
resposta_LLM = rqa(inconsistencias[10])

In [175]:
resposta_LLM['result']

' False. The budgetary savings due to underutilization of funds are not necessarily a fact in this context, as the context does not discuss the budget and the savings that could be achieved by underutilizing the funds. Therefore, this fact is false.'

In [172]:
resposta_LLM['source_documents'][0]

Document(page_content='5i) Ressaltou primeiramente que foi possível constatar que de fato em algumas despesas \ninformadas pelo gestor ocorreram erros na classificação do Código de Aplicação. \nii) Em relação aos valores questionados informou que do montante de R$ 709.040,75 \nque não teriam sido considerados incialmente, apenas parte dessas despesas R$ \n493.916,33, devem ser consideradas, as quais, apesar de classificadas na fonte do \nTesouro, foram pagas por recursos de convênios vinculados à Educação. \nii) Quanto aos demais aspectos relacionados a possíveis divergências nos Restos a \nPagar alegados pela defesa, a Unidade Técnica manteve os valores incialmente \napurados, tendo em vista terem sido apurados pelo anexo XI, enviado pela gestão, via \nSAGRES WEB.\niii) Quanto aos outros valores divergentes entre a planilha da DFAM e a apresentada \npela Defesa, manteve-se o inicialmente apurado. \niv) Os cálculos foram refeitos, considerando os valores das despesas acima \nmencionada

In [143]:
resposta_LLM['source_documents'][1]

Document(page_content='3Defesa: Justificou que tal ato foi um equívoco do setor do planejamento que ao elaborar o projeto de \nLei Orçamentário para exercício de 2017, considerou 1% (um por cento) da Receita Corrente. Disse que \nhouve a correção do vício de natureza formal.\nAnálise Técnica: Esclareceu que a Reserva de Contingência é uma dotação que não é especificamente \ndestinada a uma Unidade Orçamentária, cujos recursos serão utilizados para abertura de créditos \nadicionais. Nesse caso a defesa apenas reconheceu o limite ultrapassado.\n1.4 – Atraso de 23 dias no envio do SAGRES Contábil referente ao mês de dezembro de \n2017.\nDefesa: Alegou que o atraso não deveria ser considerado, pois a Administração teria pedido o \ncancelamento de um primeiro envio e que o reenvio teria ocorrido dentro do prazo de 10 dias \nestipulados pela Resolução TCE-PI nº 27/2016.\nAnálise Técnica: Informou que a referida prestação de contas foi enviada em 01/03/2018, \nporém cancelada a pedido do gest

In [ ]:

==> CARREGAR O ARQUVIO

wrapped_text = textwrap.fill(article, 
                             width=100,
                             break_long_words=False,
                             replace_whitespace=False)
print(wrapped_text)

In [8]:
llm = OpenAI(temperature=0)

checker_chain = LLMSummarizationCheckerChain(llm=llm, 
                                             verbose=True, 
                                             max_checks=2
                                             )
checker_chain.check_assertions_prompt.template

C:\Users\felip\anaconda3\lib\site-packages\langchain\chains\llm_summarization_checker\base.py:114: UserWarning: Directly instantiating an LLMSummarizationCheckerChain with an llm is deprecated. Please instantiate with sequential_chain argument or using the from_llm class method.
  warnings.warn(


'You are an expert fact checker. You have been hired by a major news organization to fact check a very important story.\n\nHere is a bullet point list of facts:\n"""\n{assertions}\n"""\n\nFor each fact, determine whether it is true or false about the subject. If you are unable to determine whether the fact is true or false, output "Undetermined".\nIf the fact is false, explain why.\n\n'

In [12]:
from PyPDF2 import PdfReader
from langchain.text_splitter import CharacterTextSplitter


def read_PDF(pdf_dir):
    doc_reader = PdfReader(pdf_dir)
    raw_text = ''
    for i, page in enumerate(doc_reader.pages):
        text = page.extract_text()
        if text:
            raw_text += text
    return raw_text

def split_text(raw_text):
    text_splitter = CharacterTextSplitter(        
        separator = "\n",
        chunk_size = 2500,
        chunk_overlap  = 400, #striding over the text
        length_function = len,
    )
    texts = text_splitter.split_text(raw_text)
    return texts

raw_text = read_PDF(r"C:\Users\felip\OneDrive - Fundacao Getulio Vargas - FGV\!_Paper04_Chat_PDF\fact_check_linha_35.pdf")
texts = split_text(raw_text)

In [14]:
len(texts)

13

In [18]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
embeddings = OpenAIEmbeddings()
#docsearch = FAISS.from_texts(texts, embeddings)      
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":2})
rqa = RetrievalQA.from_chain_type(llm=OpenAI(), 
                                  chain_type="stuff", 
                                  retriever=retriever,
                                  chain_type_kwargs={"prompt": prompt_input},
                                  return_source_documents=True)

In [23]:
inconsistencias = """Errors in the classification of expenses in the Code of Application
Discrepancies in the values questioned regarding expenses related to education
Divergences in Restos a Pagar values
Divergences between the SIOPE and SGRES Contábil regarding maintenance and development expenses
Error in the accounting of expenses for Actions and Public Health Services
Inconsistencies in the Portal of Transparency, including missing information and outdated contracts
Incomplete and non-compliant Demonstrativo de Disponibilidade de Caixa e Restos a Pagar
Deficiencies in the submission of required documents and reports
Deficit in revenue compared to the budgeted amount
Decrease in tax revenue from the previous year
Budgetary savings due to underutilization of funds
Superavit in the comparison of revenue and expenses
Delays in the submission of budgetary documents and reports to the TCE-PI"""

inconsistencias = inconsistencias.split('\n')
for i in inconsistencias:
    query = i
    print(query)

Errors in the classification of expenses in the Code of Application
Discrepancies in the values questioned regarding expenses related to education
Divergences in Restos a Pagar values
Divergences between the SIOPE and SGRES Contábil regarding maintenance and development expenses
Error in the accounting of expenses for Actions and Public Health Services
Inconsistencies in the Portal of Transparency, including missing information and outdated contracts
Incomplete and non-compliant Demonstrativo de Disponibilidade de Caixa e Restos a Pagar
Deficiencies in the submission of required documents and reports
Deficit in revenue compared to the budgeted amount
Decrease in tax revenue from the previous year
Budgetary savings due to underutilization of funds
Superavit in the comparison of revenue and expenses
Delays in the submission of budgetary documents and reports to the TCE-PI


In [24]:
checker_chain.create_assertions_prompt.template

'Given some text, extract a list of facts from the text.\n\nFormat your output as a bulleted list.\n\nText:\n"""\n{summary}\n"""\n\nFacts:'

In [25]:
inconsistence_check = """Role: You are a fact checker.
You know this context:{context}
Here is a fact: {question}
Task: determine whether the fact is true or false about the subject.
If you are unable to determine whether the fact is true or false, output "Undetermined".
If the fact is true or false, explain why.
Here is the answer:"""

prompt_inconsistence_check = PromptTemplate(template=inconsistence_check, input_variables=['context', 'question'])